In [1]:
from jupyter_auth_login import login

# Perform login
login_response = login(
    username="nandarohan442@gmail.com",
    password="Ron1506313015$",
    by_sms=True,
    store_session=False
)

# Save token to file
with open("robinhood_token.txt", "w") as f:
    f.write(login_response["access_token"])

print("✅ Token saved to robinhood_token.txt")


Verification workflow required. Please check your Robinhood app for instructions.
Waiting for challenge to be validated
5.088316440582275
Waiting for challenge to be validated
10.18752670288086
Waiting for challenge to be validated
15.273086309432983
✅ Token saved to robinhood_token.txt


In [2]:
import requests
import pandas as pd
from datetime import datetime
import os

# Load token from file
def load_token_from_file(file_path="robinhood_token.txt"):
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            return f.read().strip()
    return None

# Fetch and save stock data
def fetch_and_save_stock_data(token, output_file="my_stock_data.xlsx"):
    url = "https://api.robinhood.com/positions/"
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print("Error fetching stock data:", response.text)
        return

    positions = response.json().get("results", [])
    stock_data = []

    # Get the current timestamp
    fetch_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    for position in positions:
        if float(position["quantity"]) > 0:  # Only fetch open positions
            instrument_url = position["instrument"]
            instrument_data = requests.get(instrument_url, headers=headers).json()

            symbol = instrument_data["symbol"]
            quantity = float(position["quantity"])
            average_buy_price = float(position["average_buy_price"])
            quote_url = f"https://api.robinhood.com/quotes/{symbol}/"
            current_price = float(requests.get(quote_url, headers=headers).json()["last_trade_price"])

            market_value = current_price * quantity
            profit_loss = (current_price - average_buy_price) * quantity

            stock_data.append({
                "Timestamp": fetch_time,  # Add the timestamp
                "Symbol": symbol,
                "Quantity": quantity,
                "Average Buy Price": average_buy_price,
                "Current Price": current_price,
                "Market Value": market_value,
                "Profit/Loss": profit_loss
            })

    df = pd.DataFrame(stock_data)
    df.to_excel(output_file, index=False)
    print(f"Stock data saved to {output_file}")

# Use token to fetch data
token = load_token_from_file()
if token:
    fetch_and_save_stock_data(token)
else:
    print("No token found. Please log in first.")


Stock data saved to my_stock_data.xlsx


In [3]:
!pip install tqdm

In [4]:
import requests
import pandas as pd
import os
import time
from tqdm import tqdm  # ✅ Add this

# Function to fetch all tickers from Robinhood
def fetch_all_tickers_from_robinhood():
    classification_map = {
        "stock": "Equity",
        "etf": "Exchange-Traded Fund",
        "warrant": "Derivative",
        "mutual_fund": "Mutual Fund",
        "bond": "Fixed Income",
    }
    sector_classifications = {
        "software": "Technology",
        "hardware": "Technology",
        "semiconductors": "Technology",
        "it services": "Technology",
        "pharmaceuticals": "Healthcare",
        "biotech": "Healthcare",
        "medical devices": "Healthcare",
        "healthcare services": "Healthcare",
        "banks": "Financials",
        "insurance companies": "Financials",
        "investment firms": "Financials",
        "reits": "Real Estate",
        "retail": "Consumer Discretionary",
        "automotive": "Consumer Discretionary",
        "travel": "Consumer Discretionary",
        "entertainment": "Consumer Discretionary",
        "food": "Consumer Staples",
        "beverages": "Consumer Staples",
        "personal products": "Consumer Staples",
        "household goods": "Consumer Staples",
        "oil": "Energy",
        "gas": "Energy",
        "renewable energy": "Energy",
        "energy equipment": "Energy",
        "manufacturing": "Industrials",
        "construction": "Industrials",
        "aerospace": "Industrials",
        "logistics": "Industrials",
        "mining": "Materials",
        "chemicals": "Materials",
        "construction materials": "Materials",
        "forestry products": "Materials",
        "electricity": "Utilities",
        "water": "Utilities",
        "natural gas": "Utilities",
        "renewable utilities": "Utilities",
        "commercial reits": "Real Estate",
        "residential reits": "Real Estate",
        "property management": "Real Estate",
        "telecommunications": "Communication Services",
        "media": "Communication Services",
        "internet services": "Communication Services",
        "cloud services": "Technology",
        "ai": "Technology"
    }

    url = "https://api.robinhood.com/instruments/"
    tickers = []
    next_url = url

    print("⏳ Fetching data pages from Robinhood...")
    with tqdm(desc="Pages fetched", unit="page") as pbar:
        while next_url:
            try:
                response = requests.get(next_url)
                response.raise_for_status()
                data = response.json()
                tickers.extend([
                    {
                        "Ticker": item['symbol'],
                        "Name": item['name'],
                        "Type": item['type'],
                        "Classification": classification_map.get(item['type'], "Unknown"),
                        "Sector": sector_classifications.get(item['name'].lower(), "Unknown"),
                        "Tradeable": item['tradeable']
                    } for item in data.get('results', [])
                ])
                next_url = data.get('next')
                pbar.update(1)  # ✅ increment the loading bar

                if next_url:
                    time.sleep(1)  # Respect rate limits
            except requests.exceptions.RequestException as e:
                print(f"❌ Error fetching tickers: {e}")
                break

    return tickers

# Save tickers to an Excel file
def save_tickers_to_excel(tickers, filename=None):
    if filename is None:
        downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")
        filename = os.path.join(downloads_path, "robinhood_tickers.xlsx")
    df = pd.DataFrame(tickers)
    df = df[df['Tradeable'] == True]  # Filter out non-tradable tickers
    df.to_excel(filename, index=False)
    print(f"✅ Tickers saved to {filename}")

# Main logic
def main():
    print("🚀 Starting ticker scrape from Robinhood...")
    tickers = fetch_all_tickers_from_robinhood()

    if not tickers:
        print("⚠️ No tickers fetched. Exiting...")
        return

    save_tickers_to_excel(tickers)

if __name__ == "__main__":
    main()


🚀 Starting ticker scrape from Robinhood...
⏳ Fetching data pages from Robinhood...


Pages fetched: 250page [05:29,  1.30s/page]

✅ Tickers saved to C:\Users\gargi\Downloads\robinhood_tickers.xlsx


In [5]:
pip install yfinance pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import yfinance as yf
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import Javascript, display

# --- Load tickers, filtering only stocks ---
def load_tickers_from_excel(file_path):
    try:
        df = pd.read_excel(file_path)
        stock_df = df[
            df['Type'].astype(str).str.lower().str.contains('stock', na=False) &
            (df['Tradeable'] == True)
        ]
        print("🧠 Sample stock tickers:", stock_df['Ticker'].head(10).tolist())
        return stock_df['Ticker'].tolist()
    except Exception as e:
        print(f"❌ Error loading tickers from Excel: {e}")
        return []

# --- Clean ticker symbols ---
def clean_tickers(tickers):
    return [str(ticker).strip().upper() for ticker in tickers if isinstance(ticker, str) and ticker.strip()]

# --- Validate a single ticker with 7-day lookback ---
def validate_ticker(ticker):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="7d")  # ✅ Looser check (works on weekends)
        if not hist.empty:
            return ticker, True
    except Exception:
        pass
    return ticker, False

# --- Validate using parallel threads ---
def validate_tickers_parallel(tickers, max_workers=10):
    valid_tickers = []
    invalid_tickers = []

    print(f"🔍 Validating {len(tickers)} tickers using {max_workers} threads...")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(validate_ticker, ticker): ticker for ticker in tickers}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Validation progress"):
            ticker, is_valid = future.result()
            if is_valid:
                valid_tickers.append(ticker)
            else:
                invalid_tickers.append(ticker)

    print(f"✅ Valid tickers: {len(valid_tickers)}, ❌ Invalid tickers: {len(invalid_tickers)}")
    return valid_tickers, invalid_tickers

# --- Save filtered tickers to Excel ---
def save_filtered_tickers(valid_tickers, invalid_tickers, output_file):
    valid_df = pd.DataFrame({'Valid Tickers': valid_tickers})
    invalid_df = pd.DataFrame({'Invalid Tickers': invalid_tickers})

    with pd.ExcelWriter(output_file) as writer:
        valid_df.to_excel(writer, sheet_name='Valid Tickers', index=False)
        invalid_df.to_excel(writer, sheet_name='Invalid Tickers', index=False)

    print(f"📁 Filtered tickers saved to {output_file}")

# --- Run next cell in Jupyter Notebook ---
def run_next_cell():
    try:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
    except:
        print("⚠️ Unable to run next cell — likely not in classic Jupyter Notebook.")

# --- Main execution function ---
def main():
    input_file = "robinhood_tickers.xlsx"
    output_file = "validated_tickers.xlsx"

    print("📥 Loading tickers from Excel...")
    tickers = load_tickers_from_excel(input_file)

    if not tickers:
        print("⚠️ No tickers found. Exiting...")
        return

    print("🧹 Cleaning ticker list...")
    tickers = clean_tickers(tickers)

    if not tickers:
        print("⚠️ No valid tickers after cleaning. Exiting...")
        return

    valid_tickers, invalid_tickers = validate_tickers_parallel(tickers)

    print("💾 Saving results...")
    save_filtered_tickers(valid_tickers, invalid_tickers, output_file)

    print("✅ Done. Moving to next cell (if supported)...")
    run_next_cell()

# --- Run the pipeline ---
if __name__ == "__main__":
    main()


📥 Loading tickers from Excel...
🧠 Sample stock tickers: ['JCSE', 'ARTV', 'ALTS', 'SNWV', 'WRLD', 'CSAI', 'CE', 'CRH', 'GLNG', 'TSQ']
🧹 Cleaning ticker list...
🔍 Validating 5185 tickers using 10 threads...


Failed to get ticker 'SNWV' reason: Expecting value: line 1 column 1 (char 0)                 | 0/5185 [00:00<?, ?it/s]
ALTS: No price data found, symbol may be delisted (period=7d)
ARTV: No price data found, symbol may be delisted (period=7d)
Failed to get ticker 'CSAI' reason: Expecting value: line 1 column 1 (char 0)         | 1/5185 [00:00<57:10,  1.51it/s]
WRLD: No price data found, symbol may be delisted (period=7d)
JCSE: No price data found, symbol may be delisted (period=7d)
CE: No price data found, symbol may be delisted (period=7d)
GLNG: No price data found, symbol may be delisted (period=7d)
CRH: No price data found, symbol may be delisted (period=7d)                          | 6/5185 [00:01<12:25,  6.95it/s]
SNWV: No price data found, symbol may be delisted (period=7d)
TSQ: No price data found, symbol may be delisted (period=7d)
APXIF: No price data found, symbol may be delisted (period=7d)
LGF.B: No price data found, symbol may be delisted (period=7d)                      

✅ Valid tickers: 0, ❌ Invalid tickers: 5185
💾 Saving results...
📁 Filtered tickers saved to validated_tickers.xlsx
✅ Done. Moving to next cell (if supported)...


<IPython.core.display.Javascript object>

In [ ]:
def main():
    input_file = "robinhood_tickers.xlsx"
    output_file = "validated_tickers.xlsx"

    print("📥 Loading tickers from Excel...")
    tickers = load_tickers_from_excel(input_file)

    print("🔍 Sample tickers before cleaning:", tickers[:10])  # 👀 Show raw tickers

    if not tickers:
        print("⚠️ No tickers found. Exiting...")
        return

    print("🧹 Cleaning ticker list...")
    tickers = clean_tickers(tickers)

    print("🧼 Cleaned tickers:", tickers[:10])  # 👀 Show cleaned tickers

    valid_tickers, invalid_tickers = validate_tickers_parallel(tickers)

    print("💾 Saving results...")
    save_filtered_tickers(valid_tickers, invalid_tickers, output_file)

    print("✅ Done.")

# Now call it to test
main()


📥 Loading tickers from Excel...
🔍 Sample tickers before cleaning: ['JCSE', 'ARTV', 'ALTS', 'SNWV', 'WRLD', 'CSAI', 'CE', 'CRH', 'GLNG', 'TSQ']
🧹 Cleaning ticker list...
🧼 Cleaned tickers: ['JCSE', 'ARTV', 'ALTS', 'SNWV', 'WRLD', 'CSAI', 'CE', 'CRH', 'GLNG', 'TSQ']
🔍 Validating 5185 tickers using 10 threads...


ARTV: No price data found, symbol may be delisted (period=7d)                                 | 0/5185 [00:00<?, ?it/s]
JCSE: No price data found, symbol may be delisted (period=7d)                         | 1/5185 [00:00<30:27,  2.84it/s]
ALTS: No price data found, symbol may be delisted (period=7d)
WRLD: No price data found, symbol may be delisted (period=7d)
Failed to get ticker 'SNWV' reason: Expecting value: line 1 column 1 (char 0)
CE: No price data found, symbol may be delisted (period=7d)
Failed to get ticker 'CSAI' reason: Expecting value: line 1 column 1 (char 0)
CRH: No price data found, symbol may be delisted (period=7d)
GLNG: No price data found, symbol may be delisted (period=7d)
TSQ: No price data found, symbol may be delisted (period=7d)                          | 7/5185 [00:00<08:13, 10.48it/s]
APXIF: No price data found, symbol may be delisted (period=7d)
LGF.B: No price data found, symbol may be delisted (period=7d)
SWKH: No price data found, symbol may be delisted (

✅ Valid tickers: 0, ❌ Invalid tickers: 5185
💾 Saving results...
📁 Filtered tickers saved to validated_tickers.xlsx
✅ Done.


In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import time
import requests
from concurrent.futures import ThreadPoolExecutor
import yfinance.shared

# Create a custom session with a timeout
def set_request_timeout(timeout=10):
    session = requests.Session()
    session.request = lambda *args, **kwargs: requests.Session.request(session, *args, timeout=timeout, **kwargs)
    yfinance.shared._session = session

# Set the timeout for requests
set_request_timeout(10)

# Load validated tickers from Excel
def load_validated_tickers(file_path):
    try:
        df = pd.read_excel(file_path, sheet_name='Valid Tickers')
        return df['Valid Tickers'].tolist()
    except Exception as e:
        print(f"Error loading tickers from Excel: {e}")
        return []

# Batch download historical data in smaller chunks with retry logic and delays
def batch_download(tickers, period="1mo", batch_size=50, max_retries=3):
    all_data = []
    total_batches = len(tickers) // batch_size + (1 if len(tickers) % batch_size > 0 else 0)
    failed_tickers = []

    for i in range(0, len(tickers), batch_size):
        batch = tickers[i:i + batch_size]
        print(f"Downloading batch {i // batch_size + 1} of {total_batches}...")
        batch_failures = []
        for attempt in range(max_retries):
            try:
                data = yf.download(batch, period=period, group_by="ticker", threads=True, progress=False)
                all_data.append(data)
                break  # Exit retry loop on success
            except Exception as e:
                print(f"Attempt {attempt + 1} failed for batch {i // batch_size + 1}: {e}")
                time.sleep(2 ** attempt)  # Exponential backoff
                if attempt == max_retries - 1:  # If max retries reached
                    batch_failures.extend(batch)
        print(f"Batch {i // batch_size + 1} completed: {len(batch) - len(batch_failures)} successful, {len(batch_failures)} failed.")
        failed_tickers.extend(batch_failures)
        time.sleep(5)  # Delay between batches

    if failed_tickers:
        print(f"Failed to download data for {len(failed_tickers)} tickers.")
        save_failed_tickers(failed_tickers)

    if all_data:
        return pd.concat(all_data, axis=1)
    else:
        print("No data downloaded.")
        return None

# Save failed tickers to Excel
def save_failed_tickers(failed_tickers, file_name="failed_tickers.xlsx"):
    pd.DataFrame({'Failed Tickers': failed_tickers}).to_excel(file_name, index=False)
    print(f"Failed tickers saved to {file_name}")

# Filter tickers by liquidity
def filter_by_liquidity(data, min_volume=100000, min_dollar_volume=1000000):
    filtered_tickers = []
    for ticker in data.columns.get_level_values(0).unique():
        try:
            hist = data[ticker]
            avg_volume = hist['Volume'].mean()
            avg_dollar_volume = (hist['Close'] * hist['Volume']).mean()

            if avg_volume >= min_volume and avg_dollar_volume >= min_dollar_volume:
                filtered_tickers.append(ticker)
        except Exception as e:
            print(f"Liquidity check failed for {ticker}: {e}")

    print(f"Tickers after liquidity check: {len(filtered_tickers)}")
    return filtered_tickers

# Filter tickers by volatility
def filter_by_volatility(data, max_volatility=0.05):
    filtered_tickers = []
    for ticker in data.columns.get_level_values(0).unique():
        try:
            hist = data[ticker]
            returns = hist['Close'].pct_change()
            avg_volatility = returns.std()

            if avg_volatility <= max_volatility:
                filtered_tickers.append(ticker)
        except Exception as e:
            print(f"Volatility check failed for {ticker}: {e}")

    print(f"Tickers after volatility check: {len(filtered_tickers)}")
    return filtered_tickers

# Save filtered tickers to Excel
def save_filtered_tickers(filtered_tickers, output_file):
    filtered_df = pd.DataFrame({'Filtered Tickers': filtered_tickers})

    with pd.ExcelWriter(output_file) as writer:
        filtered_df.to_excel(writer, sheet_name='Filtered Tickers', index=False)

    print(f"Filtered tickers saved to {output_file}")

# Main function
def main():
    input_file = "validated_tickers.xlsx"  # Input file with validated tickers
    output_file = "filtered_tickers.xlsx"  # Output file for filtered tickers
    period = "1mo"  # Set your desired period
    batch_size = 50  # Number of tickers per batch

    print("Loading validated tickers...")
    tickers = load_validated_tickers(input_file)

    if not tickers:
        print("No validated tickers found. Exiting...")
        return

    print("Downloading historical data for tickers in batches...")
    data = batch_download(tickers, period=period, batch_size=batch_size)

    if data is None:
        print("Failed to download data. Exiting...")
        return

    print("Filtering by liquidity...")
    liquid_tickers = filter_by_liquidity(data)

    print("Filtering by volatility...")
    final_tickers = filter_by_volatility(data[liquid_tickers])

    print("Saving filtered tickers...")
    save_filtered_tickers(final_tickers, output_file)

if __name__ == "__main__":
    main()


Loading validated tickers...
No validated tickers found. Exiting...
